<a href="https://colab.research.google.com/github/mystakhs/test-Self-Supervised-Learning/blob/main/DINOv2_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DINOv2の動作確認
## 1. 必要パッケージインストール

In [ ]:
!pip install timm

## 2. インポート

In [ ]:
# 基本パッケージ
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 画像処理・データセット関連
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

# 評価用
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity as sklearn_cosine_similarity
from collections import Counter

# 可視化・ログ
import matplotlib.pyplot as plt
import pandas as pd

# DINOv2のBackbone用
import timm

# デバイス設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## 3. DINOv2に合わせたモデル定義
※DINOv2はProjectorを内部に持つので、別途Projectorクラスは不要。

ここではtimmから**dinov2_vits14**をロードして特徴抽出専用に使用。

In [ ]:
# DINOv2 Feature Extractor
class DINOv2Extractor(nn.Module):
    def __init__(self, model_name="vit_small", out_dim=384):
        super().__init__()
        self.backbone = timm.create_model('vit_small_patch14_dinov2.lvd142m', pretrained=True)
        self.out_dim = out_dim

    def forward(self, x):
        features = self.backbone.forward_features(x)  # ここで特徴量だけ取り出す
        return features


## 4. データセット
※SimCLR時とほぼ同様。ただし、DINOv2は解像度が最低でも224x224推奨。

In [ ]:
# DINOv2用データセット（CIFAR10 224x224リサイズ）
class DINOv2Dataset(Dataset):
    def __init__(self, root, transform, train=True):
        self.dataset = datasets.CIFAR10(root=root, train=train, download=True)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        image = self.transform(image)
        return image, label


## 5. データ拡張・ローダー


In [ ]:
# データ拡張（DINOv2推奨に近いもの）
transform = transforms.Compose([
    transforms.Resize(518),
    transforms.CenterCrop(518),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# データローダー
train_dataset = DINOv2Dataset(root='./data', transform=transform, train=True)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

val_dataset = DINOv2Dataset(root='./data', transform=transform, train=False)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2)


## 6. 事前定義された損失関数（DINOv2ではContrastive不要）
今回は、DINOv2の特徴を直接k-NN評価。
そのため、自己教師あり事前学習済みモデルをそのまま使い、損失関数不要。

## 7. 特徴量抽出・k-NN

In [ ]:
# 特徴量抽出
def feature_extraction(model, data_loader):
    model.eval()
    all_features = []
    all_labels = []

    with torch.no_grad():
        for x, y in data_loader:
            # x = x.to(device, dtype=torch.float16)  # ← 省メモリ化 (float16)
            x = x.to(device)  # ★float16変換しない！！
            y = y.to(device)

            features = model(x)
            # features = features.float()  # 安全のため float32に戻す（optional）

            # 特徴量プーリング（DINOv2は出力が(バッチ, パッチ数+1, hidden_dim)）
            if features.ndim == 3:
                features = features[:, 0]  # CLSトークンだけ取る

            all_features.append(features.cpu())
            all_labels.append(y.cpu())

    # メモリ圧縮したまま結合
    features = torch.cat(all_features, dim=0).numpy()
    labels = torch.cat(all_labels, dim=0).numpy()

    return features, labels


# k-NN分類（コサイン類似度）
def knn_cosine(train_features, train_labels, val_features, k=200):
    cosine_sim = sklearn_cosine_similarity(val_features, train_features)

    val_label_pred = []
    for i in range(val_features.shape[0]):
        top_k_indices = np.argsort(-cosine_sim[i])[:k]
        top_k_labels = train_labels[top_k_indices]
        most_common_label = Counter(top_k_labels).most_common(1)[0][0]
        val_label_pred.append(most_common_label)

    return np.array(val_label_pred)


## 8. モデル・評価・可視化

In [ ]:
## モデル
model = DINOv2Extractor().to(device)

# 特徴量取得
train_features, train_labels = feature_extraction(model, train_loader)
val_features, val_labels = feature_extraction(model, val_loader)

# k-NN
predictions = knn_cosine(train_features, train_labels, val_features, k=200)
knn_acc = accuracy_score(val_labels, predictions)
print(f'k-NN Validation Accuracy: {knn_acc:.2f}')

# 混同行列
cm = confusion_matrix(val_labels, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=val_dataset.dataset.classes)
disp.plot(xticks_rotation='vertical')
plt.title("Confusion Matrix")
plt.show()

# t-SNE可視化
tsne = TSNE(n_components=2, random_state=0, perplexity=30)
val_features_2d = tsne.fit_transform(val_features)

plt.figure(figsize=(8, 8))
scatter = plt.scatter(val_features_2d[:, 0], val_features_2d[:, 1], c=val_labels, cmap='tab10', alpha=0.7)
plt.legend(*scatter.legend_elements(), title="Classes", bbox_to_anchor=(1.05, 1))
plt.title("t-SNE Visualization")
plt.grid(True)
plt.show()


上記はfeature_extraction()でメモリオーバーでクラッシュする

# roboflowでの実装例

In [ ]:
!pip install torch numpy opencv-python scikit-learn roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.

In [ ]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image


In [ ]:
dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14")
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
dinov2_vits14.to(device)

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vits14_pretrain.pth
100%|██████████| 84.2M/84.2M [00:00<00:00, 92.2MB/s]


DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [ ]:
transform_image = T.Compose([
    T.ToTensor(),
    T.Resize(244),
    T.CenterCrop(224),
    T.Normalize([0.5], [0.5])
])


In [ ]:
def load_image(img_path):
    img = Image.open(img_path)
    transformed_img = transform_image(img)[:3].unsqueeze(0)
    return transformed_img

def compute_embeddings(files):
    all_embeddings = {}
    with torch.no_grad():
        for file in files:
            embeddings = dinov2_vits14(load_image(file).to(device))
            all_embeddings[file] = embeddings.cpu().numpy().reshape(1, -1).tolist()
    return all_embeddings


In [ ]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

# 画像の取得
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

# モデルとプロセッサのロード
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')

# 前処理と推論
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
# last_hidden_states = outputs.last_hidden_state
last_hidden_states = outputs[0]


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
last_hidden_states = outputs[0]
# We have to force return_dict=False for tracing
model.config.return_dict = False

with torch.no_grad():
    traced_model = torch.jit.trace(model, [inputs.pixel_values])
    traced_outputs = traced_model(inputs.pixel_values)

print((last_hidden_states - traced_outputs[0]).abs().max())

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
/usr/local/lib/python3.11/dist-packages/transformers/models/dinov2/modeling_dinov2.py:165: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:


tensor(0., grad_fn=<MaxBackward1>)


In [ ]:
print((last_hidden_states - traced_outputs[0]).abs().max())

tensor(0., grad_fn=<MaxBackward1>)


In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
from transformers import AutoImageProcessor, Dinov2ForImageClassification
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image", trust_remote_code=True)
image = dataset["test"]["image"][0]

image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-small-imagenet1k-1-layer")
model = Dinov2ForImageClassification.from_pretrained("facebook/dinov2-small-imagenet1k-1-layer")

inputs = image_processor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

cats-image.py:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/58.3k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/91.3M [00:00<?, ?B/s]

tabby, tabby cat
